In [3]:
%matplotlib inline

# python libraties
import os, cv2,itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
from glob import glob
from PIL import Image

# pytorch libraries
import torch
from torch import optim,nn
from torch.autograd import Variable
from torch.utils.data import DataLoader,Dataset
from torchvision import models,transforms

# sklearn libraries
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# ensure results are reproducible
np.random.seed(10)
torch.manual_seed(10)
torch.cuda.manual_seed(10)

In [4]:
data_dir = "/kaggle/input/aiml-general-championship"
print(os.listdir(data_dir))

['KCDH2024_Training_LesionGroupings.csv', 'KCDH2024_Test_Input', 'sample_submission.csv', 'KCDH2024_Training_Input_10K', 'KCDH2024_Training_GroundTruth.csv']


<h1>Loading Data... </h1>

In [5]:
img_data_dir = data_dir + r"/KCDH2024_Training_Input_10K/KCDH2024_Training_Input_10K"
all_image_path = glob(os.path.join(img_data_dir, '*.jpg'))
imageid_path_dict = {os.path.splitext(os.path.basename(x))[0]: x for x in all_image_path}  # key - image fil ename,   value - path to image
lesion_type_dict = {
    'nv': 'Melanocytic nevi',
    'mel': 'dermatofibroma',
    'bkl': 'Benign keratosis-like lesions ',
    'bcc': 'Basal cell carcinoma',
    'akiec': 'Actinic keratoses',
    'vasc': 'Vascular lesions',
    'df': 'Dermatofibroma'
}

In [6]:
len(imageid_path_dict)

9600

In [7]:
img = cv2.imread(imageid_path_dict["ISIC_0024308"])
img.shape

(450, 600, 3)

In [8]:
def compute_img_mean_std(image_paths):
    """
        computing the mean and std of three channel on the whole dataset,
        first we should normalize the image from 0-255 to 0-1
    """

    img_h, img_w = 224, 224              # Size to resize..
    imgs = []
    means, stdevs = [], []

    for i in tqdm(range(len(image_paths))):
        img = cv2.imread(image_paths[i])
        img = cv2.resize(img, (img_h, img_w))
        imgs.append(img)

    imgs = np.stack(imgs, axis=3)             
    print(imgs.shape)

    imgs = imgs.astype(np.float32) / 255.

    for i in range(3):
        pixels = imgs[:, :, i, :].ravel()  # resize to one row
        means.append(np.mean(pixels))
        stdevs.append(np.std(pixels))

    means.reverse()  # BGR --> RGB
    stdevs.reverse()

    print("normMean = {}".format(means))
    print("normStd = {}".format(stdevs))
    return means,stdevs

In [9]:
# means , stdevs = compute_img_mean_std(all_image_path)

In [10]:
# Store Values to save time in future..
norm_mean = [0.76696384, 0.54525656, 0.56884694]
norm_std = [0.13945772, 0.15192385, 0.16916788]

In [11]:
lesion_db = pd.read_csv(os.path.join(data_dir, 'KCDH2024_Training_LesionGroupings.csv'))
truth_db = pd.read_csv(os.path.join(data_dir, 'KCDH2024_Training_GroundTruth.csv'))

df = pd.merge(lesion_db, truth_db, on = 'image', how = 'inner')
df.drop("diagnosis_confirm_type", axis = 1, inplace = True)
df['path'] = df['image'].map(imageid_path_dict.get)
df

,image,lesion_id,MEL,NV,BCC,AKIEC,BKL,DF,VASC,path
0,ISIC_0024306,HAM_0000550,0,1,0,0,0,0,0,/kaggle/input/aiml-general-championship/KCDH20...
1,ISIC_0024307,HAM_0003577,0,1,0,0,0,0,0,/kaggle/input/aiml-general-championship/KCDH20...
2,ISIC_0024308,HAM_0001477,0,1,0,0,0,0,0,/kaggle/input/aiml-general-championship/KCDH20...
3,ISIC_0024309,HAM_0000484,0,1,0,0,0,0,0,/kaggle/input/aiml-general-championship/KCDH20...
4,ISIC_0024310,HAM_0003350,1,0,0,0,0,0,0,/kaggle/input/aiml-general-championship/KCDH20...
...,...,...,...,...,...,...,...,...,...,...
10010,ISIC_0034316,HAM_0004304,1,0,0,0,0,0,0,None
10011,ISIC_0034317,HAM_0006376,1,0,0,0,0,0,0,None
10012,ISIC_0034318,HAM_0000344,0,0,0,0,1,0,0,None
10013,ISIC_0034319,HAM_0000747,0,1,0,0,0,0,0,None


In [12]:
# Remove the rows not containg path to images..
df = df[df['path'].notna()]
df

,image,lesion_id,MEL,NV,BCC,AKIEC,BKL,DF,VASC,path
0,ISIC_0024306,HAM_0000550,0,1,0,0,0,0,0,/kaggle/input/aiml-general-championship/KCDH20...
1,ISIC_0024307,HAM_0003577,0,1,0,0,0,0,0,/kaggle/input/aiml-general-championship/KCDH20...
2,ISIC_0024308,HAM_0001477,0,1,0,0,0,0,0,/kaggle/input/aiml-general-championship/KCDH20...
3,ISIC_0024309,HAM_0000484,0,1,0,0,0,0,0,/kaggle/input/aiml-general-championship/KCDH20...
4,ISIC_0024310,HAM_0003350,1,0,0,0,0,0,0,/kaggle/input/aiml-general-championship/KCDH20...
...,...,...,...,...,...,...,...,...,...,...
9595,ISIC_0033901,HAM_0002342,1,0,0,0,0,0,0,/kaggle/input/aiml-general-championship/KCDH20...
9596,ISIC_0033902,HAM_0000048,1,0,0,0,0,0,0,/kaggle/input/aiml-general-championship/KCDH20...
9597,ISIC_0033903,HAM_0003367,0,1,0,0,0,0,0,/kaggle/input/aiml-general-championship/KCDH20...
9598,ISIC_0033904,HAM_0005820,0,1,0,0,0,0,0,/kaggle/input/aiml-general-championship/KCDH20...


In [13]:
# Convert 7 different columns of different lesisons to single ...

cell_type_idx = []

for index, row in df.iterrows():
    cell_type_idx_row = row["MEL"], row["NV"], row["BCC"], row["AKIEC"], row["BKL"], row["DF"], row["VASC"]
    cell_type_idx.append(cell_type_idx_row.index(1))

# Assign a new column..
df["cell_type_idx"] = cell_type_idx   

# Drop older columns..
df.drop( columns = ["MEL", "NV", "BCC", "AKIEC", "BKL", "DF", "VASC"], inplace = True)

df.head()

/tmp/ipykernel_34/2182716373.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["cell_type_idx"] = cell_type_idx
/tmp/ipykernel_34/2182716373.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop( columns = ["MEL", "NV", "BCC", "AKIEC", "BKL", "DF", "VASC"], inplace = True)


,image,lesion_id,path,cell_type_idx
0,ISIC_0024306,HAM_0000550,/kaggle/input/aiml-general-championship/KCDH20...,1
1,ISIC_0024307,HAM_0003577,/kaggle/input/aiml-general-championship/KCDH20...,1
2,ISIC_0024308,HAM_0001477,/kaggle/input/aiml-general-championship/KCDH20...,1
3,ISIC_0024309,HAM_0000484,/kaggle/input/aiml-general-championship/KCDH20...,1
4,ISIC_0024310,HAM_0003350,/kaggle/input/aiml-general-championship/KCDH20...,0


In [14]:
# Determine how many images are associated with each lesion_id ?
df_undup = df.groupby('lesion_id').count()

# Filter out lesion_id's that have only one image associated with it
df_undup = df_undup[df_undup['image'] == 1]
df_undup.reset_index(inplace=True)
df_undup.head()

,lesion_id,image,path,cell_type_idx
0,HAM_0000001,1,1,1
1,HAM_0000003,1,1,1
2,HAM_0000004,1,1,1
3,HAM_0000007,1,1,1
4,HAM_0000008,1,1,1


In [15]:
df_undup = df.groupby('lesion_id').count()
df_undup.head()

# Filter out lesion_id's that have only one image associated with it
df_undup = df_undup[df_undup['image'] == 1]
df_undup.reset_index(inplace=True)
df_undup[:]

,lesion_id,image,path,cell_type_idx
0,HAM_0000001,1,1,1
1,HAM_0000003,1,1,1
2,HAM_0000004,1,1,1
3,HAM_0000007,1,1,1
4,HAM_0000008,1,1,1
...,...,...,...,...
5592,HAM_0007622,1,1,1
5593,HAM_0007623,1,1,1
5594,HAM_0007624,1,1,1
5595,HAM_0007626,1,1,1


In [16]:
# Identify lesion_id's that have duplicate images and those that have only one image.
def get_duplicates(x):
    unique_list = list(df_undup['lesion_id'])
    if x in unique_list:
        return 'unduplicated'
    else:
        return 'duplicated'

# Create a new colum that is a copy of the lesion_id column
df['duplicates'] = df['lesion_id']
# Apply the function to this new column
df['duplicates'] = df['duplicates'].apply(get_duplicates)
df.head()

/tmp/ipykernel_34/1363690245.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['duplicates'] = df['lesion_id']
/tmp/ipykernel_34/1363690245.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['duplicates'] = df['duplicates'].apply(get_duplicates)


,image,lesion_id,path,cell_type_idx,duplicates
0,ISIC_0024306,HAM_0000550,/kaggle/input/aiml-general-championship/KCDH20...,1,unduplicated
1,ISIC_0024307,HAM_0003577,/kaggle/input/aiml-general-championship/KCDH20...,1,unduplicated
2,ISIC_0024308,HAM_0001477,/kaggle/input/aiml-general-championship/KCDH20...,1,unduplicated
3,ISIC_0024309,HAM_0000484,/kaggle/input/aiml-general-championship/KCDH20...,1,unduplicated
4,ISIC_0024310,HAM_0003350,/kaggle/input/aiml-general-championship/KCDH20...,0,duplicated


In [17]:
df['duplicates'].value_counts()

duplicates
unduplicated    5597
duplicated      4003
Name: count, dtype: int64

In [18]:
# Filter out images that don't have duplicates (count = 1)  (Removed augmented images)
# We will use this data to get validation set..
df_undup = df[df['duplicates'] == 'unduplicated']
df_undup.shape

(5597, 5)

In [19]:
# Create a val set using df as none of these images have augmented duplicates in the training set now..
y = df_undup['cell_type_idx']
_, df_val = train_test_split(df_undup, test_size=0.2, random_state=101, stratify=y)
df_val.shape

(1120, 5)

In [20]:
df_val['cell_type_idx'].value_counts()

cell_type_idx
1    891
4     89
0     52
2     37
3     30
6     13
5      8
Name: count, dtype: int64

In [21]:
# Remove the validation rows from the original data to get training rows..

# This function identifies if an image is part of the train or val set.
def get_val_rows(x):
    # create a list of all the lesion_id's in the val set
    val_list = list(df_val['image'])
    if str(x) in val_list:
        return 'val'
    else:
        return 'train'

# Identify train and val rows..
# Create a new colum that is a copy of the image column
df['train_or_val'] = df['image']

# Apply the function to this new column
df['train_or_val'] = df['train_or_val'].apply(get_val_rows)

# Filter out training rows
df_train = df[df['train_or_val'] == 'train']
df_train = df_train.drop('train_or_val', axis = 1, inplace = False)
print(len(df_train))
print(len(df_val))

/tmp/ipykernel_34/2742417337.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['train_or_val'] = df['image']


8480
1120


/tmp/ipykernel_34/2742417337.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['train_or_val'] = df['train_or_val'].apply(get_val_rows)


In [22]:
df_train['cell_type_idx'].value_counts()

cell_type_idx
1    5566
4     966
0     963
2     458
3     297
6     125
5     105
Name: count, dtype: int64

In [23]:
# Duplicate rows to balance the number of rows in 7 classes..
data_aug_rate = [5,0,10,15,5,50,40]

# Iterate over unique values of 'cell_type_idx'
for i in df_train['cell_type_idx'].unique():

    if data_aug_rate[i]:
        
        # Filter the DataFrame for the current value of 'cell_type_idx'
        filtered_df = df_train[df_train['cell_type_idx'] == i]
        
        # Duplicate rows based on the data augmentation rate for this value of 'cell_type_idx'
        duplicated_rows = filtered_df.sample(frac=data_aug_rate[i] - 1, replace=True)
        
        # Concatenate the original DataFrame with the duplicated rows
        df_train = pd.concat([df_train, duplicated_rows], ignore_index=True)

df_train['cell_type_idx'].value_counts()

cell_type_idx
1    5566
5    5250
6    5000
4    4830
0    4815
2    4580
3    4455
Name: count, dtype: int64

<h1>DataSet Spliiting</h1>

In [24]:
# Split the test set again in a validation set and a true test set:

df_val, df_test = train_test_split(df_val, test_size=0.5)
df_train = df_train.reset_index()
df_val = df_val.reset_index()
df_test = df_test.reset_index()

In [25]:
print(len(df_test))
df_test['cell_type_idx'].value_counts()

560


cell_type_idx
1    440
4     53
0     24
2     19
3     17
6      4
5      3
Name: count, dtype: int64

<h1>Define Model</h1>

In [26]:
# feature_extract is a boolean that defines finetuning or feature extracting. 
# If feature_extract = False, the model is finetuned and all model parameters are updated. 
# If feature_extract = True, only the last layer parameters are updated, the others remain fixed.

def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

In [27]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18, resnet34, resnet50, resnet101
        """
        model_ft = models.resnet50(pretrained=True)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "mobilenet":
        model_ft = models.mobilenet_v2(pretrained=True, progress=True)
        set_parameter_requires_grad(model_ft, feature_extract)
        input_size = 224

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

In [28]:
# Define a new model variable..

model_name = "mobilenet"
num_classes = 7
feature_extract = False
# Initialize the model
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Define the device:
device = torch.device('cuda:0')
# device = torch.device('cpu') # If using cpu
# Put the model on the device:
model = model_ft.to(device)

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 112MB/s] 


<H1>Augmenting Data...</H1>

In [29]:
# define the transformation of the train images.
train_transform = transforms.Compose([transforms.Resize((input_size,input_size)),transforms.RandomHorizontalFlip(),
                                      transforms.RandomVerticalFlip(),
                                      transforms.RandomRotation(20),
                                      transforms.ColorJitter(brightness=0.1, contrast=0.1, hue=0.1),
                                      transforms.ToTensor(), 
                                      transforms.Normalize(norm_mean, norm_std)])

# define the transformation of the val images.
val_transform = transforms.Compose([transforms.Resize((input_size,input_size)), 
                                    transforms.ToTensor(),
                                    transforms.Normalize(norm_mean, norm_std)])

# define the transformation of the test images.
test_transform = transforms.Compose([transforms.Resize((input_size,input_size)), 
                                     transforms.ToTensor(),
                                    transforms.Normalize(norm_mean, norm_std)])

In [30]:
# Define a pytorch dataloader for dataset..
class HAM10000(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        # Load data and get label
        X = Image.open(self.df['path'][index])
        y = torch.tensor(int(self.df['cell_type_idx'][index]))

        if self.transform:
            X = self.transform(X)

        return X, y

In [31]:
# Define the training set using the table train_df and using the defined transitions (train_transform)
training_set = HAM10000(df_train, transform=train_transform)
train_loader = DataLoader(training_set, batch_size=32, shuffle=True, num_workers=0)

# Same for the validation set:
validation_set = HAM10000(df_val, transform=train_transform)
val_loader = DataLoader(validation_set, batch_size=32, shuffle=False, num_workers=0)

# Same for the test set:
test_set = HAM10000(df_test, transform=train_transform)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False, num_workers=0)

In [32]:
# set optimizer and loss function
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss().to(device)

In [33]:
# this function is used during training process, to calculate the loss and accuracy
class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [34]:
total_loss_train, total_acc_train = [],[]

def train(train_loader, model, criterion, optimizer, epoch):
    model.train()
    train_loss = AverageMeter()
    train_acc = AverageMeter()
    curr_iter = (epoch - 1) * len(train_loader)
    for i, data in enumerate(train_loader):
        
        images, labels = data
        N = images.size(0)
        images = Variable(images).to(device)
        labels = Variable(labels).to(device)

        optimizer.zero_grad()
        outputs = model(images)

        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        prediction = outputs.max(1, keepdim=True)[1]
        train_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)
        train_loss.update(loss.item())
        curr_iter += 1
        if (i + 1) % 100 == 0:
            print('[epoch %d], [iter %d / %d], [train loss %.5f], [train acc %.5f]' % (
                epoch, i + 1, len(train_loader), train_loss.avg, train_acc.avg))
            total_loss_train.append(train_loss.avg)
            total_acc_train.append(train_acc.avg)
    return train_loss.avg, train_acc.avg

In [35]:
def validate(val_loader, model, criterion, optimizer, epoch):
    model.eval()
    val_loss = AverageMeter()
    val_acc = AverageMeter()
    
    with torch.no_grad():
        for i, data in tqdm(enumerate(val_loader)):
            images, labels = data
            N = images.size(0)
            images = Variable(images).to(device)
            labels = Variable(labels).to(device)

            outputs = model(images)
            prediction = outputs.max(1, keepdim=True)[1]

            val_acc.update(prediction.eq(labels.view_as(prediction)).sum().item()/N)
            val_loss.update(criterion(outputs, labels).item())

    print('------------------------------------------------------------')
    print('[epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, val_loss.avg, val_acc.avg))
    print('------------------------------------------------------------')
    return val_loss.avg, val_acc.avg

In [42]:
# Start Training..

epoch_num = 10
best_val_acc = 0
total_loss_val, total_acc_val = [],[]
for epoch in tqdm(range(1, epoch_num+1)):
    loss_train, acc_train = train(train_loader, model, criterion, optimizer, epoch)
    loss_val, acc_val = validate(val_loader, model, criterion, optimizer, epoch)
    total_loss_val.append(loss_val)
    total_acc_val.append(acc_val)
    if acc_val > best_val_acc:
        best_val_acc = acc_val
        print('*****************************************************')
        print('best record: [epoch %d], [val loss %.5f], [val acc %.5f]' % (epoch, loss_val, acc_val))
        print('*****************************************************')

  0%|          | 0/10 [00:00<?, ?it/s]

[epoch 1], [iter 100 / 1078], [train loss 1.60715], [train acc 0.54312]
[epoch 1], [iter 200 / 1078], [train loss 1.27424], [train acc 0.59516]
[epoch 1], [iter 300 / 1078], [train loss 1.12017], [train acc 0.62979]
[epoch 1], [iter 400 / 1078], [train loss 1.03159], [train acc 0.65312]
[epoch 1], [iter 500 / 1078], [train loss 0.97033], [train acc 0.66950]
[epoch 1], [iter 600 / 1078], [train loss 0.92006], [train acc 0.68370]
[epoch 1], [iter 700 / 1078], [train loss 0.88075], [train acc 0.69634]
[epoch 1], [iter 800 / 1078], [train loss 0.84895], [train acc 0.70668]
[epoch 1], [iter 900 / 1078], [train loss 0.82254], [train acc 0.71424]
[epoch 1], [iter 1000 / 1078], [train loss 0.79945], [train acc 0.72062]



0it [00:00, ?it/s]
1it [00:01,  1.08s/it]
2it [00:01,  1.03it/s]
3it [00:02,  1.08it/s]
4it [00:03,  1.11it/s]
5it [00:04,  1.13it/s]
6it [00:05,  1.15it/s]
7it [00:06,  1.15it/s]
8it [00:07,  1.18it/s]
9it [00:07,  1.17it/s]
10it [00:08,  1.14it/s]
11it [00:09,  1.18it/s]
12it [00:10,  1.22it/s]
13it [00:11,  1.21it/s]
14it [00:12,  1.18it/s]
15it [00:12,  1.18it/s]
16it [00:13,  1.20it/s]
17it [00:14,  1.17it/s]
18it [00:15,  1.19it/s]
 10%|█         | 1/10 [11:19<1:41:53, 679.22s/it]

------------------------------------------------------------
[epoch 1], [val loss 0.61614], [val acc 0.79514]
------------------------------------------------------------
*****************************************************
best record: [epoch 1], [val loss 0.61614], [val acc 0.79514]
*****************************************************
[epoch 2], [iter 100 / 1078], [train loss 0.53757], [train acc 0.79906]
[epoch 2], [iter 200 / 1078], [train loss 0.54916], [train acc 0.79641]
[epoch 2], [iter 300 / 1078], [train loss 0.54587], [train acc 0.79656]
[epoch 2], [iter 400 / 1078], [train loss 0.53761], [train acc 0.79922]
[epoch 2], [iter 500 / 1078], [train loss 0.53634], [train acc 0.79794]
[epoch 2], [iter 600 / 1078], [train loss 0.53337], [train acc 0.80036]
[epoch 2], [iter 700 / 1078], [train loss 0.52737], [train acc 0.80192]
[epoch 2], [iter 800 / 1078], [train loss 0.52617], [train acc 0.80312]
[epoch 2], [iter 900 / 1078], [train loss 0.52344], [train acc 0.80299]
[epoch 2], 


0it [00:00, ?it/s]
1it [00:00,  2.18it/s]
2it [00:00,  2.17it/s]
3it [00:01,  2.21it/s]
4it [00:01,  2.20it/s]
5it [00:02,  2.18it/s]
6it [00:02,  2.17it/s]
7it [00:03,  2.17it/s]
8it [00:03,  2.18it/s]
9it [00:04,  2.18it/s]
10it [00:04,  2.19it/s]
11it [00:05,  2.20it/s]
12it [00:05,  2.17it/s]
13it [00:05,  2.19it/s]
14it [00:06,  2.18it/s]
15it [00:06,  2.15it/s]
16it [00:07,  2.16it/s]
17it [00:07,  2.16it/s]
18it [00:08,  2.24it/s]
 20%|██        | 2/10 [20:44<1:21:38, 612.31s/it]

...

------------------------------------------------------------
[epoch 8], [val loss 0.31210], [val acc 0.89931]
------------------------------------------------------------
*****************************************************
best record: [epoch 8], [val loss 0.31210], [val acc 0.89931]
*****************************************************
[epoch 9], [iter 100 / 1078], [train loss 0.28584], [train acc 0.89281]
[epoch 9], [iter 200 / 1078], [train loss 0.28200], [train acc 0.89422]
[epoch 9], [iter 300 / 1078], [train loss 0.27560], [train acc 0.89750]
[epoch 9], [iter 400 / 1078], [train loss 0.27571], [train acc 0.89742]
[epoch 9], [iter 500 / 1078], [train loss 0.27383], [train acc 0.89900]
[epoch 9], [iter 600 / 1078], [train loss 0.26966], [train acc 0.89943]
[epoch 9], [iter 700 / 1078], [train loss 0.26619], [train acc 0.90138]
[epoch 9], [iter 800 / 1078], [train loss 0.26556], [train acc 0.90145]
[epoch 9], [iter 900 / 1078], [train loss 0.26343], [train acc 0.90205]
[epoch 9], 


0it [00:00, ?it/s]
1it [00:00,  2.17it/s]
2it [00:00,  2.17it/s]
3it [00:01,  2.19it/s]
4it [00:01,  2.07it/s]
5it [00:02,  2.10it/s]
6it [00:02,  2.12it/s]
7it [00:03,  2.12it/s]
8it [00:03,  2.15it/s]
9it [00:04,  2.14it/s]
10it [00:04,  2.13it/s]
11it [00:05,  2.16it/s]
12it [00:05,  2.18it/s]
13it [00:06,  2.19it/s]
14it [00:06,  2.17it/s]
15it [00:06,  2.17it/s]
16it [00:07,  2.17it/s]
17it [00:07,  2.17it/s]
18it [00:08,  2.22it/s]
 90%|█████████ | 9/10 [1:26:59<09:28, 568.94s/it]

------------------------------------------------------------
[epoch 9], [val loss 0.42071], [val acc 0.86979]
------------------------------------------------------------
[epoch 10], [iter 100 / 1078], [train loss 0.24622], [train acc 0.91500]
[epoch 10], [iter 200 / 1078], [train loss 0.23084], [train acc 0.91516]
[epoch 10], [iter 300 / 1078], [train loss 0.23238], [train acc 0.91344]
[epoch 10], [iter 400 / 1078], [train loss 0.24496], [train acc 0.90867]
[epoch 10], [iter 500 / 1078], [train loss 0.24173], [train acc 0.91019]
[epoch 10], [iter 600 / 1078], [train loss 0.24207], [train acc 0.90948]
[epoch 10], [iter 700 / 1078], [train loss 0.24097], [train acc 0.90929]
[epoch 10], [iter 800 / 1078], [train loss 0.24159], [train acc 0.90879]
[epoch 10], [iter 900 / 1078], [train loss 0.24086], [train acc 0.90938]
[epoch 10], [iter 1000 / 1078], [train loss 0.24208], [train acc 0.90919]



0it [00:00, ?it/s]
1it [00:00,  2.17it/s]
2it [00:00,  2.19it/s]
3it [00:01,  2.21it/s]
4it [00:01,  2.18it/s]
5it [00:02,  2.20it/s]
6it [00:02,  2.21it/s]
7it [00:03,  2.19it/s]
8it [00:03,  2.17it/s]
9it [00:04,  2.15it/s]
10it [00:04,  2.17it/s]
11it [00:05,  2.19it/s]
12it [00:05,  2.19it/s]
13it [00:05,  2.19it/s]
14it [00:06,  2.19it/s]
15it [00:06,  2.18it/s]
16it [00:07,  2.18it/s]
17it [00:07,  2.15it/s]
18it [00:08,  2.24it/s]
100%|██████████| 10/10 [1:36:25<00:00, 578.59s/it]

------------------------------------------------------------
[epoch 10], [val loss 0.36628], [val acc 0.86632]
------------------------------------------------------------


In [46]:
# Save the model's state dictionary to a file
torch.save(model.state_dict(), 'classification_model.pth')